In [1]:
# packages needed for this tutorial
import pathlib
# math and robotics
import numpy as np
import spatialmath as sm
import quaternion
from tqdm.notebook import tqdm, trange

# plotting 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt

# simulated robot content and helpers
import hrr_common
import hrr_cobot_robot

# ROS
import rospy

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib notebook

from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal
from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
rospy.init_node('Restore_Defaults_2_Notebook')

In [3]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")

In [4]:
def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()

In [ ]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

### Open and CLose tool changer / careful

In [ ]:
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

## Calibrate - Asssume Skill Server Running

In [6]:
cobot.change_tool("wsg_50_dsa")

True

In [ ]:
cobot.T_B_E_robot, cobot.T_B_C_robot

In [8]:
cobot.R_FT_E, sm.base.rotz(-45,"deg")

(array([[ 0.70711,  0.70711,  0.     ],
        [-0.70711,  0.70711,  0.     ],
        [ 0.     ,  0.     ,  1.     ]]),
 array([[ 0.70711,  0.70711,  0.     ],
        [-0.70711,  0.70711,  0.     ],
        [ 0.     ,  0.     ,  1.     ]]))

In [12]:
q_calib2 = np.r_[1.34109, 0.09328, -1.44435, -0.01627, 1, -0.00009]
cobot.move_to_joint_pose(q_calib2, stochastic=False)

In [ ]:
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = True
goal.scale_q4 = 0.3
goal.scale_q5 = 0.2
goal.scale_q6 = 0.3
goal.q4_dot_des = 0.2
goal.q5_dot_des = 0.2
goal.q6_dot_des = 0.2
goal.keep_current_pose=True
goal

In [ ]:
calibrate_client.send_goal(goal)

In [ ]:
cancel(calibrate_client)

In [ ]:
# get client result / feedback
get_result(calibrate_client)

In [ ]:
# plot results -> copy line to terminal
print(f"please run the following line in a separate terminal:\nrosrun hrr_cobot_robot plot_ft_calibration -p {goal.calibration_file} {goal.data_file}")

In [ ]:
cobot.update()
cobot.B_F_msr, cobot.FT_F, cobot.FT.f,cobot.FT.wrench_calib, cobot.FT.wrench, cobot.FT.offset

In [ ]:
rospy.sleep(5)
np.r_[quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[0:3]), quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[3:6])]

In [ ]:
#This is how B_F_msr works:
def quat_rotate_vec(q1, v):
    return (q1 * np.quaternion(0, *v) * q1.conjugate()).vec
np.r_[quat_rotate_vec(cobot.sns_quat, cobot.R_FT_E.T @ cobot.FT_F[0:3]), quat_rotate_vec(cobot.sns_quat, cobot.R_FT_E.T @ cobot.FT_F[3:6])]


In [ ]:
np.r_[quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[0:3]), quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[3:6])]

In [20]:
cobot.move_to_joint_pose(cobot.q_calib)

In [ ]:
cobot._ft_handle.offset

## Compliance

In [ ]:
cobot.init_sns_vel()

In [ ]:
cobot.reset()
cobot.set_py_hybrid_force_vel_command(B_F_des=np.r_[0.0, 0.0, -5, np.zeros(3)], 
                                          K_f=8e-4, K_t=5e-3, scale_pos=0.0, scale_rot=0.0, 
                                      wrench_dir=np.r_[1.0, 1.0, 1.0, np.ones(3)],
                                      vel_dir=np.zeros(6))

In [ ]:
T = int(10 * cobot.hz)
for t in trange(T):
    cobot.update()

In [ ]:
cobot.emergency_stop()
print(cobot.state)
cobot.reset()
print(cobot.state)